In [19]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)


In [ ]:
df = pd.read_csv('data.csv', sep=';')

In [29]:
df['Cod_cmd'].value_counts().count()

404939

In [31]:
# Get the most common Univers and Nature for each product
product_categories = df.groupby('Libellé produit').agg({
    'Univers': lambda x: x.mode().iat[0] if not x.mode().empty else None,
    'Nature': lambda x: x.mode().iat[0] if not x.mode().empty else None
}).reset_index()

df_unique_products = product_categories[['Libellé produit', 'Univers', 'Nature']]

print("Number of unique products:", len(df_unique_products))
df_unique_products.head()


Number of unique products: 56018


,Libellé produit,Univers,Nature
0,40 742 ensemble 5 en 1 tondeuse et regle barb...,Petit Electroménager,Tondeuse multistyle
1,60cm fronton applique mural style arbre de vi...,Décoration Textile,Objet deco
2,9 vip pets vague 1,Enfant Bébé,Poupée et poupon
3,agère murale hwc h37 étagère suspendue tiroir...,Canapé Salon Séjour,Etagère
4,allan canapé 3 places convertible ouverture e...,Petit Electroménager,Friteuse


In [1]:
df_unique_products.to_csv('unique_products.csv', index=False)

NameError: name 'df_unique_products' is not defined

In [17]:
df = pd.read_csv('unique_products.csv')
df.sample(10)

,Libellé produit,Univers,Nature
45186,Sommier tapissier 130x190 gris fabrique en france,Chambre Literie,Sommier
47604,Table basse jeny 115 x 65 x 40 cm blanc,Canapé Salon Séjour,Table basse
16637,Etagère à chaussures superposable 6 paires 67 x 31 cm noir,Bureau Rangement,Meuble à chaussures
25835,Lit enfant anna m 70x140cm +tiroir+matelas+sommier,Enfant Bébé,Lit jeune
18290,Fmd bureau avec 1 tiroir 123 x 50 x 76 5 cm béton et blanc,Bureau Rangement,Bureau
21538,Icaverne fixations et meubles de rangement multimédia serie armoire à cd noir brillant 21x20x88 cm aggloméré,Jardin Loisirs Sport,Visserie consommable
36159,Oreiller duvet plumettes 65x65 cm,Chambre Literie,Oreiller
22893,Klarstein chateaubriand nuovo appareil à raclette pour 8 grill 1200w pierre in,Petit Electroménager,Appareil à raclette
16917,FURNIX meuble tv élégant 100 cm Blois I blanc noir avec LED,Canapé Salon Séjour,Meuble tv
10723,Chevet bois teck japonais 2 tiroirs,Chambre Literie,Chevet


In [4]:
df['Nature'].value_counts().count()

594

In [ ]:
df[df['Nature'] == 'Colonne sdb']['Libellé produit']

In [5]:
import sys, subprocess
subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "tokenizers", "scikit-learn"])


0

In [6]:
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.normalizers import NFD, Lowercase, StripAccents, Sequence
from tokenizers.processors import TemplateProcessing
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import numpy as np
import pandas as pd
import os

# Ensure df is loaded (from earlier cells). Expect columns: 'Libellé produit', 'Nature'
assert 'df' in globals(), "DataFrame df must be loaded earlier."
assert {'Libellé produit','Nature'}.issubset(df.columns), "df must have 'Libellé produit' and 'Nature' columns"

# Use the deduplicated products if available
if 'df_unique_products' in globals():
    data_df = df_unique_products.rename(columns={'Libellé produit': 'label', 'Nature': 'target'})
else:
    data_df = df.rename(columns={'Libellé produit': 'label', 'Nature': 'target'})[['label','target']].dropna()

# Basic cleaning: drop rows with missing or empty labels/targets
data_df = data_df.dropna(subset=['label','target'])
data_df = data_df[(data_df['label'].astype(str).str.strip()!='') & (data_df['target'].astype(str).str.strip()!='')]

# Prepare corpus for BPE
corpus = data_df['label'].astype(str).tolist()

# Initialize BPE tokenizer
bpe_tokenizer = Tokenizer(BPE(unk_token="[UNK]"))
bpe_tokenizer.normalizer = Sequence([NFD(), Lowercase(), StripAccents()])
bpe_tokenizer.pre_tokenizer = Whitespace()

# Special tokens and training
special_tokens = ["[PAD]", "[UNK]"]
trainer = BpeTrainer(vocab_size=20000, min_frequency=2, special_tokens=special_tokens)

bpe_tokenizer.train_from_iterator(corpus, trainer=trainer)

# Post-processor for consistent BOS/EOS if desired (not necessary for bag-of-tokens)
bpe_tokenizer.post_processor = TemplateProcessing(
    single="$0",
    pair="$A $B",
    special_tokens=[("[PAD]", 0), ("[UNK]", 1)]
)

# Save to disk to reuse later
bpe_dir = "./artifacts"
os.makedirs(bpe_dir, exist_ok=True)
bpe_path = os.path.join(bpe_dir, "bpe_tokenizer.json")
bpe_tokenizer.save(bpe_path)

print(f"Trained BPE tokenizer with vocab size: {bpe_tokenizer.get_vocab_size()} and saved to {bpe_path}")





Trained BPE tokenizer with vocab size: 20000 and saved to ./artifacts/bpe_tokenizer.json


In [ ]:
# Build vocabulary mapping from tokenizer
vocab = bpe_tokenizer.get_vocab()
# Invert mapping to get id->token if needed
id_to_token = {idx: tok for tok, idx in vocab.items()}

# Helper: encode to BPE token ids, ignoring specials
SPECIAL_IDS = {vocab.get("[PAD]"), vocab.get("[UNK]")}

def bpe_tokenize(text: str):
    if not isinstance(text, str):
        text = str(text) if text is not None else ""
    encoding = bpe_tokenizer.encode(text)
    # Return tokens (strings) excluding specials
    tokens = [t for t in encoding.tokens if vocab.get(t) not in SPECIAL_IDS]
    return tokens

# Create bag-of-BPE-token counts
from collections import Counter

all_tokens = []
for t in data_df['label'].tolist():
    all_tokens.extend(bpe_tokenize(t))

# Select top-k tokens to form features
from collections import Counter
freqs = Counter(all_tokens)
max_features = 50000
most_common_tokens = [tok for tok, _ in freqs.most_common(max_features)]
feature_index = {tok: i for i, tok in enumerate(most_common_tokens)}

import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix, vstack

rows, cols, vals = [], [], []
for row_idx, text in enumerate(data_df['label'].tolist()):
    counts = Counter(bpe_tokenize(text))
    for tok, c in counts.items():
        col_idx = feature_index.get(tok)
        if col_idx is not None:
            rows.append(row_idx)
            cols.append(col_idx)
            vals.append(c)

X = csr_matrix((vals, (rows, cols)), shape=(len(data_df), len(feature_index)), dtype=np.float64)
y = data_df['target'].astype(str).values

# Robust split handling rare classes (count < 2)
counts = pd.Series(y).value_counts()
freq_classes = counts[counts >= 2].index
frequent_mask = np.isin(y, freq_classes)
idx_frequent = np.where(frequent_mask)[0]
idx_rare = np.where(~frequent_mask)[0]

if len(idx_frequent) >= 2:
    X_f = X[idx_frequent]
    y_f = y[idx_frequent]
    try:
        X_train_f, X_test, y_train_f, y_test = train_test_split(
            X_f, y_f, test_size=0.1, random_state=42, stratify=y_f
        )
    except ValueError:
        X_train_f, X_test, y_train_f, y_test = train_test_split(
            X_f, y_f, test_size=0.1, random_state=42, shuffle=True, stratify=None
        )
    # Add all rare-class samples to training set
    if len(idx_rare) > 0:
        X_train = vstack([X_train_f, X[idx_rare]])
        y_train = np.concatenate([y_train_f, y[idx_rare]])
    else:
        X_train, y_train = X_train_f, y_train_f
else:
    # Fallback: no stratification possible
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.1, random_state=42, shuffle=True, stratify=None
    )

# Fit Multinomial Naive Bayes
nb = MultinomialNB(alpha=0.1)
nb.fit(X_train, y_train)

print('Train size:', X_train.shape, 'Test size:', X_test.shape)
print('Classes:', nb.classes_.shape)
print('Test accuracy:', nb.score(X_test, y_test))
print(classification_report(y_test, nb.predict(X_test)))


In [8]:
from typing import Dict

# Prepare structures for prediction
class_names = nb.classes_.tolist()
class_index = {c: i for i, c in enumerate(class_names)}


def vectorize_labels(texts):
    rows, cols, vals = [], [], []
    for row_idx, text in enumerate(texts):
        counts = Counter(bpe_tokenize(text))
        for tok, c in counts.items():
            col_idx = feature_index.get(tok)
            if col_idx is not None:
                rows.append(row_idx)
                cols.append(col_idx)
                vals.append(c)
    return csr_matrix((vals, (rows, cols)), shape=(len(texts), len(feature_index)), dtype=np.float64)


def predict_nature_proba(label_text: str) -> Dict[str, float]:
    Xq = vectorize_labels([label_text])
    proba = nb.predict_proba(Xq)[0]
    return {cls: float(prob) for cls, prob in zip(class_names, proba)}

# Example usage
example_label = data_df['label'].iloc[0]
probs = predict_nature_proba(example_label)
print(example_label)
# Display top-10 predicted Natures
for cls, p in sorted(probs.items(), key=lambda kv: kv[1], reverse=True)[:10]:
    print(f"{cls}: {p:.4f}")


 40 742 ensemble 5 en 1 tondeuse et regle barbe rechargeable
Tondeuse multistyle: 0.9889
Coiffeuse: 0.0053
Accessoire pem: 0.0015
Accessoire ch adulte: 0.0010
Petit meuble séjour: 0.0007
Ens table chaises: 0.0006
Evier: 0.0003
Chevet: 0.0003
Bar: 0.0002
Rangement bureau: 0.0001


In [9]:
import numpy as np
import pandas as pd

# Train on full data to score items
nb_full = MultinomialNB(alpha=0.1)
nb_full.fit(X, y)

class_names_full = nb_full.classes_.tolist()


def score_items_and_find_mislabels(
    df_labels_targets: pd.DataFrame,
    prob_threshold: float = 0.2,
    topk: int = 3,
) -> pd.DataFrame:
    """
    Identify potential mislabelled items by scoring each label with the trained model.

    - prob_threshold: if model's probability for the current label's target is below this, flag as candidate.
    - topk: include top-k alternative class suggestions and their probabilities.
    """
    texts = df_labels_targets['label'].astype(str).tolist()
    X_all = vectorize_labels(texts)
    proba_all = nb_full.predict_proba(X_all)
    pred_all = nb_full.predict(X_all)

    # Map true target index in classes
    class_index_full = {c: i for i, c in enumerate(class_names_full)}

    rows = []
    for i, (label_text, true_cls) in enumerate(zip(df_labels_targets['label'], df_labels_targets['target'])):
        probs = proba_all[i]
        true_idx = class_index_full.get(str(true_cls), None)
        true_prob = float(probs[true_idx]) if true_idx is not None else np.nan
        top_indices = np.argsort(probs)[::-1][:topk]
        top_suggestions = [(class_names_full[j], float(probs[j])) for j in top_indices]
        is_mislabel = (not np.isnan(true_prob)) and (true_prob < prob_threshold)
        rows.append({
            'label': label_text,
            'true_nature': true_cls,
            'true_prob': true_prob,
            'pred_nature': pred_all[i],
            'pred_prob': float(np.max(probs)),
            'top_suggestions': top_suggestions,
            'candidate_mislabel': bool(is_mislabel),
        })

    result = pd.DataFrame(rows)
    return result.sort_values(['candidate_mislabel','true_prob'], ascending=[False, True]).reset_index(drop=True)


mislabel_df = score_items_and_find_mislabels(data_df, prob_threshold=0.2, topk=5)
print('Candidates:', mislabel_df['candidate_mislabel'].sum(), 'of', len(mislabel_df))
mislabel_path = './artifacts/mislabel_candidates.csv'
mislabel_df.to_csv(mislabel_path, index=False)
print('Saved to', mislabel_path)

# Show a preview of flagged items
mislabel_df[mislabel_df['candidate_mislabel']].head(10)


Candidates: 4874 of 56012
Saved to ./artifacts/mislabel_candidates.csv


,label,true_nature,true_prob,pred_nature,pred_prob,top_suggestions,candidate_mislabel
0,Meuble demi colonne meuble bas de salle de bai...,Cabine de douche,1.970660e-17,Colonne sdb,0.999972,"[(Colonne sdb, 0.9999724038413559), (Rangement...",True
1,Pack pr t à dormir matelas ressorts ensachés +...,Meuble à chaussures,2.889092e-16,Matelas + sommier,1.000000,"[(Matelas + sommier, 1.0), (Meuble à chaussure...",True
2,Giantex ensemble table en bois avec 4 chaises ...,Accessoire ch enfant,1.177051e-15,Ens table chaises,0.899567,"[(Ens table chaises, 0.8995672803815707), (Tab...",True
3,Pack pr t à dormir matelas actimemo relax mémo...,Meuble à chaussures,1.642677e-15,Matelas + sommier,1.000000,"[(Matelas + sommier, 1.0), (Meuble à chaussure...",True
4,Tapis dessous de tapis asma tapis de salon sha...,Objet deco,2.190956e-15,Tapis de Salon et Ch,1.000000,"[(Tapis de Salon et Ch, 1.0), (Objet deco, 2.1...",True
5,Pack pr t à dormir matelas actimemo tech 180x2...,Meuble à chaussures,3.197709e-15,Matelas + sommier,1.000000,"[(Matelas + sommier, 1.0), (Meuble à chaussure...",True
6,Ambiante drap housse pour surmatelas en coton ...,Meuble à chaussures,3.865459e-15,Drap housse,0.999997,"[(Drap housse, 0.9999967789543127), (Drap plat...",True
7,Songmics étagère à 5 niveaux 200 x 100 x 50 cm...,Carport,9.109821e-15,Etagère,1.000000,"[(Etagère, 0.999999999993662), (Meuble à chaus...",True
8,Lot de 4 chaises scandinaves chaises design ch...,Fauteuil,1.200938e-14,Chaise,1.000000,"[(Chaise, 0.9999999993829363), (Table, 4.00152...",True
9,Table de bar table haute de bar mange debout c...,Linge de table,2.739860e-14,Table,0.761638,"[(Table, 0.7616382900704377), (Bar, 0.23832970...",True


In [21]:
from sklearn.metrics import accuracy_score, precision_score
from sklearn.model_selection import StratifiedKFold
from collections import Counter
from scipy.sparse import csr_matrix
import numpy as np
import pandas as pd

# 1) Hold-out test metrics from the earlier split
y_pred_test = nb.predict(X_test)
acc_test = accuracy_score(y_test, y_pred_test)
prec_macro_test = precision_score(y_test, y_pred_test, average='macro', zero_division=0)
prec_weighted_test = precision_score(y_test, y_pred_test, average='weighted', zero_division=0)
print('Hold-out test metrics:')
print('  Accuracy       :', round(acc_test, 4))
print('  Precision (macro)  :', round(prec_macro_test, 4))
print('  Precision (weighted):', round(prec_weighted_test, 4))

# 2) Stratified 5-fold CV with per-fold feature index (to reduce leakage)
labels_all = data_df['label'].astype(str).values
targets_all = data_df['target'].astype(str).values

# Remove classes with <2 samples for stratified CV
class_counts = pd.Series(targets_all).value_counts()
freq_classes_cv = class_counts[class_counts >= 2].index
mask_cv = np.isin(targets_all, freq_classes_cv)
labels_cv = labels_all[mask_cv]
y_cv = targets_all[mask_cv]

max_features_cv = 50000

def build_feature_index(texts):
    token_counter = Counter()
    for t in texts:
        token_counter.update(bpe_tokenize(t))
    most_common = [tok for tok, _ in token_counter.most_common(max_features_cv)]
    return {tok: i for i, tok in enumerate(most_common)}

def vectorize_with_index(texts, feat_idx):
    rows, cols, vals = [], [], []
    for row_idx, t in enumerate(texts):
        counts = Counter(bpe_tokenize(t))
        for tok, c in counts.items():
            col_idx = feat_idx.get(tok)
            if col_idx is not None:
                rows.append(row_idx)
                cols.append(col_idx)
                vals.append(c)
    Xloc = csr_matrix((vals, (rows, cols)), shape=(len(texts), len(feat_idx)), dtype=np.float64)
    return Xloc

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
accs, prec_macros, prec_weighteds = [], [], []

for fold_idx, (tr_idx, va_idx) in enumerate(skf.split(labels_cv, y_cv), 1):
    labels_tr, labels_va = labels_cv[tr_idx], labels_cv[va_idx]
    y_tr, y_va = y_cv[tr_idx], y_cv[va_idx]

    feat_idx = build_feature_index(labels_tr)
    X_tr = vectorize_with_index(labels_tr, feat_idx)
    X_va = vectorize_with_index(labels_va, feat_idx)

    clf = MultinomialNB(alpha=0.1)
    clf.fit(X_tr, y_tr)
    y_va_pred = clf.predict(X_va)

    accs.append(accuracy_score(y_va, y_va_pred))
    prec_macros.append(precision_score(y_va, y_va_pred, average='macro', zero_division=0))
    prec_weighteds.append(precision_score(y_va, y_va_pred, average='weighted', zero_division=0))

print('\n5-fold CV metrics (mean +/- std):')
print('  Accuracy       :', f"{np.mean(accs):.4f} +/- {np.std(accs):.4f}")
print('  Precision (macro)  :', f"{np.mean(prec_macros):.4f} +/- {np.std(prec_macros):.4f}")
print('  Precision (weighted):', f"{np.mean(prec_weighteds):.4f} +/- {np.std(prec_weighteds):.4f}")


Hold-out test metrics:
  Accuracy       : 0.7512
  Precision (macro)  : 0.5513
  Precision (weighted): 0.7287


/home/valentin/.local/lib/python3.10/site-packages/sklearn/model_selection/_split.py:805: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(



5-fold CV metrics (mean +/- std):
  Accuracy       : 0.7426 +/- 0.0042
  Precision (macro)  : 0.5568 +/- 0.0126
  Precision (weighted): 0.7284 +/- 0.0058


In [26]:
import pandas as pd

df = pd.read_csv('unique_products.csv', sep=',')
df.sample(1000).to_csv('unique_products_sample.csv', index=False)

In [1]:
import pandas as pd

df = pd.read_csv('artifacts/unique_products_extracted.csv', sep=',')
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
df.sample(100)
# .to_csv('result_sample.csv', index=False)

,Libellé produit,Univers,Nature,extracted_dimension,extracted_colors
18593,Friteuse comelec fr 3070 3071 3 3 l 3000w acier inoxydable,Petit Electroménager,Accessoire pem,NaN,NaN
37207,Parasol déporté carré 3x3m hardelot 3x3m gris manivelle anti retour toile déperlante facile à utiliser,Jardin Loisirs Sport,Parasol,3 * 300,gris
19272,Giantex table de cuisine avec 2 chaises pour salle à manger en couleur du bois dimension de la table 80x53x75 cm l x l x h ca,Cuisine Salle de bain,Ens table chaises,80 * 53 * 75,NaN
46379,Sur meuble bureau enfant ugo,Bureau Rangement,Bureau,NaN,NaN
18494,Four micro ondes teka tmw 18 bi noir,Gros Electroménager,Micro ondes,NaN,noir
53750,Ventilateur de plafond luminaire 2 en 1,Décoration Textile,Objets lumineux,NaN,NaN
13009,Coussin rectangulaire ethnique à franges 25 x 58 cm,Décoration Textile,Coussin,25 * 58,NaN
13468,Cuve ronde en inox,Cuisine Salle de bain,Evier,NaN,NaN
49375,Tabouret coffre valise patchwork multicolore,Canapé Salon Séjour,Fauteuil,NaN,"multicolore, multicolor"
28037,Lot de 2 taies d oreillers percale blanc 50x70,Décoration Textile,Taie d'oreiller,50 * 70,blanc
